In [ ]:
//export
//default_exp ModB
//package ComposedExample

# Import the module we defined in ToImport.ipynb renamed to -> ModA.sc
> Using nbdev, we exported the notebook contents from `ToImport.ipynb` to the Scala script `ModA.sc`. Now we can import the script to this notebook by calling: 
```Scala
import $file.^.nbdev.ModA, ModA._
```

In [ ]:
//export
import $file.^.nbdev.ModA, ModA._

### Call the load_ivy script.
Located in the source folder, the load_ivy.sc script will use ammonite to download snapshots of the chisel libraries, to be cached on your system. This is necessary for EVERY chisel notebook so we can then import our chisel libraries like we would in an SBT project structure.

In [ ]:
//export
import $file.^.source.load_ivy

### Prevent namespace error
- To prevent an ambiguous reference to 'Input' by chisel3.Input and _root_.almond.input.Input, we have to make sure that we rename our chisel3 Input on import. It suffices to do as follows without requiring to change the name to something like IInput:
```scala
import chisel3.{Input => Input} 
```

In [ ]:
//export
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.test

### Importing chisel modules from .sc files
To import the scripts contents as a single object do:
```scala
import $file.filename
filename.Blah()
```

To import all of the scripts contents do:
```scala
import $file.filename, filename._
Blah()
```


We can now try and test our module to see if we imported it correctly:

In [ ]:
test(new Add) { c =>
    c.io.a.poke(1.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(3.U)
    
    c.io.a.poke(5.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(7.U)
}

### Lets use this imported module to build a module in this notebook:

In [ ]:
//export
class ComposedModule extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(4.W))
        val b = Input(UInt(4.W))
        val c = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    val addr = Module(new Add)
    addr.io.a := io.a
    addr.io.b := io.b
    io.out := addr.io.out + io.c
}

In [ ]:
test(new ComposedModule) { c =>
    c.io.a.poke(1.U)    
    c.io.b.poke(2.U)    
    c.io.c.poke(3.U)    
    c.clock.step(1)    
    c.io.out.expect(6.U)
    
    c.io.a.poke(5.U)    
    c.io.b.poke(2.U)    
    c.io.c.poke(7.U)    
    c.clock.step(1)    
    c.io.out.expect(14.U)
}